## Imports

In [1]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import time
import math
import os
import urllib.request
import ptb_iterator as reader

plt.rc('figure', figsize=(10, 7))

## Load and process data

In [2]:
if False:
    file_url = 'https://raw.githubusercontent.com/jcjohnson/torch-rnn/master/data/tiny-shakespeare.txt'
    file_name = 'tinyshakespeare.txt'

    if not os.path.exists(file_name):
        urllib.request.urlretrieve(file_url, file_name)
else:
#    file_name = 'majakovski.txt'
    file_name = 'vm.txt' # Leo Tolstoy

with open(file_name, 'r') as f:
    raw_data = f.read()
    print('Data length: {} bytes'.format(len(raw_data)))

vocab = set(raw_data)
vocab_size = len(vocab)
idx_to_vocab = dict(enumerate(vocab))
vocab_to_idx = dict(zip(idx_to_vocab.values(), idx_to_vocab.keys()))

# cross-validation set size in percents of the dataset size 
cv_size = int(len(raw_data) * 0.05)

data = [vocab_to_idx[c] for c in raw_data]

del raw_data

Data length: 2978767 bytes


In [3]:
# utility functions
def gen_epochs(n, seq_len, batch_size):
    for i in range(n):
        yield reader.ptb_iterator(data, batch_size, seq_len)

In [4]:
cv_size

148938

# Build the graph

## Cell factory

In [5]:
def create_rnn_cells(cell_type, state_size, num_layers, pkeep):
    state_is_tuple = False
    
    if cell_type == 'GRU':
        single_cell = lambda: tf.contrib.rnn.GRUCell(state_size)
    elif cell_type == 'LSTM':
        single_cell = lambda: tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)
        state_is_tuple = True
    else:
        single_cell = lambda: tf.contrib.rnn.BasicRNNCell(state_size)

    # wrap a cell with dropout
    cell_creator = lambda: tf.contrib.rnn.DropoutWrapper(single_cell(), input_keep_prob=pkeep)

    cells = [cell_creator() for _ in range(num_layers)]
    return (cells, state_is_tuple)

In [6]:
# input parameters
CELLTYPE = 'GRU'
SEQLEN = 40
NCLASSES = vocab_size
NLAYERS = 3
INTERNALSIZE = 600
BATCHSIZE = 96

learning_rate = 1e-4
dropout_pkeep = 1.0
save_filename = '{0}-{1}_{2}s_{3}l_{4}is_{5}bs{6}'.format(file_name, CELLTYPE, SEQLEN,
                                                          NLAYERS, INTERNALSIZE, BATCHSIZE,
                                                          '_d' if dropout_pkeep != 1.0 else '')
print(save_filename)

vm.txt-GRU_40s_3l_600is_96bs


## The graph

In [7]:
# placeholders
lrate = tf.placeholder(tf.float32, name='learningrate')
pkeep = tf.placeholder(tf.float32, name='pkeep')
batchsize = tf.placeholder(tf.int32, name='batchsize')

# input/output
x = tf.placeholder(tf.int32, [None, None], name='input_placeholder') # [BATCHSIZE x SEQLEN]
rnn_inputs = tf.one_hot(x, NCLASSES, 1.0, 0.0) # [BATCHSIZE x SEQLEN x NUMCLASSES]

y = tf.placeholder(tf.int32, [None, None], name='labels_placeholder') # [BATCHSIZE x SEQLEN]
y_oh = tf.one_hot(y, NCLASSES, 1.0, 0.0) # [BATCHSIZE x SEQLEN x NUMCLASSES]

# RNN cells
cells, state_is_tuple = create_rnn_cells(cell_type=CELLTYPE,
                                         state_size=INTERNALSIZE,
                                         num_layers=NLAYERS,
                                         pkeep=pkeep)

# combine them to a multicell
multicell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=state_is_tuple)
multicell = tf.contrib.rnn.DropoutWrapper(multicell, input_keep_prob=pkeep)

init_state = tf.placeholder(tf.float32, [None, NLAYERS * INTERNALSIZE],
                            name='init_state') # [BATCHSIZE x NLAYERS * INTERNALSIZE]

# rnn_outputs: [BATCHSIZE x SEQLEN x INTERNALSIZE]
# final_state: [BATCHSIZE x NLAYERS * INTERNALSIZE]
rnn_outputs, final_state = tf.nn.dynamic_rnn(multicell, rnn_inputs, 
                                             dtype=tf.float32, initial_state=init_state)

# just to give it a name
final_state = tf.identity(final_state, name='final_state')

In [8]:
# softmax layer
rnn_outputs = tf.reshape(rnn_outputs, [-1, INTERNALSIZE]) # [BATCHSIZE * SEQLEN x INTERNALSIZE]
logits = tf.contrib.layers.linear(rnn_outputs, NCLASSES) # [BATCHSIZE * SEQLEN x NCLASSES]

y_reshaped = tf.reshape(y_oh, [-1, NCLASSES]) # [BATCHSIZE * SEQLEN x NCLASSES]

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped) # [BATCHSIZE x SEQLEN]
loss = tf.reshape(loss, [batchsize, -1]) # [BATCHSIZE, SEQLEN]
train_step = tf.train.AdamOptimizer(lrate).minimize(loss)

In [9]:
# TODO: rename it
Yo = tf.nn.softmax(logits) # [BATCHSIZE x SEQLEN, NCLASSES]
Y = tf.argmax(Yo, axis=1) # [BATCHSIZE x SEQLEN]
Y = tf.reshape(Y, [batchsize, -1]) # [BATCHSIZE, SEQLEN]

In [10]:
# stats for display
seqloss = tf.reduce_mean(loss, axis=1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y, tf.cast(Y, tf.int32)), tf.float32))

# summaries
loss_summary = tf.summary.scalar('batch_loss', batchloss)
acc_summary = tf.summary.scalar('batch_accuracy', accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

# Training

## Init the session

In [11]:
# summary writer & saver
timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter('log/' + save_filename + '-' + timestamp + '-training')

# save me
save_dir = 'checkpoints/'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

saver = tf.train.Saver(max_to_keep=1)

In [12]:
# init session
session = tf.Session()

# restore the session
if False:
    checkpoint = ''
    try:
        print('Trying to restore last checkpoint...')

        last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=save_dir)
        saver.restore(session, save_path=last_chk_path)

        print('Restored checkpoint from: ', last_chk_path)

    except:
        print('Failed to restore checkpoint, initializing variables instead')
        session.run(tf.global_variables_initializer())
else:
    session.run(tf.global_variables_initializer())

## Helper function to generate a sample of text

In [13]:
def generate_characters(num_chars, prompt='A', pick_top_chars=None):
    state = np.zeros([1, INTERNALSIZE * NLAYERS])
    current_char = vocab_to_idx[prompt]
    chars = [current_char]

    for i in range(num_chars):
        feed_dict = {x: [[current_char]], init_state: state, pkeep: 1.0, batchsize: 1}

        preds, state = session.run([Yo, final_state], feed_dict)

        p = np.squeeze(preds)

        if pick_top_chars is not None:
            p[np.argsort(p)[:-pick_top_chars]] = 0
            p = p / np.sum(p)

        current_char = np.random.choice(vocab_size, 1, p=p)[0]

        chars.append(current_char)

    chars = map(lambda x: idx_to_vocab[x], chars)
    return "".join(chars)

## Training

In [22]:
### training
DISPLAY_FREQ = 300
ND_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
NEPOCH = 20
step = 0
istate = np.zeros([BATCHSIZE, NLAYERS * INTERNALSIZE])

for idx, epoch in enumerate(gen_epochs(NEPOCH, SEQLEN, BATCHSIZE)):
    for xe, ye in epoch:
        if xe.shape[1] != SEQLEN or ye.shape[1] != SEQLEN:
            continue

        feed_dict = {x: xe, y: ye,
                     init_state: istate,
                     lrate: learning_rate,
                     pkeep: dropout_pkeep,
                     batchsize: BATCHSIZE
                    }
        _, y_, ostate, smm = session.run([train_step, Y, final_state, summaries], feed_dict=feed_dict)
        
        summary_writer.add_summary(smm, step)
        
        # display a short text generated with the current weights and biases
        if step % ND_BATCHES == 0:
            print('e: {0}, s: {1}: {2}'.format(idx, step,
                                               generate_characters(512, prompt=' ', pick_top_chars=7)))

        istate = ostate
        step += BATCHSIZE * SEQLEN

    print('epoch {} done'.format(idx))
    saver.save(session, save_dir + save_filename + '_' + timestamp, global_step=step)

e: 0, s: 0:  Соной и стал дальше нагайрен и, узнав, выражение сапоги и села с ней. Он очень лежал все пропановление, – что такое возморно сказать, вы не делаемое дуэло истину.
Про этот день был прислушивался к двери, у которой он спокойно и должен был вывер сторон, в присутствии Князькова, подставил подступающей в нерассказу на то место, которое он видел прежде в свои сомнения. В потерях верста от одного к англимскому бурку и убитого и разгорающегося по солнцам, загнул свое счастие дольно с постройкой на станку и спокой
e: 0, s: 1152000:  Висьма смотрели в брот плана генейать а в состоянии.. – Весь мир расправдивает смущанье, которою мамей приказывал князь Багратиона и с удовольствием вздохнул и смутился.
– Ты подходи, посмотрю, – сказал кровь. – Я думаю, не позволял детства. Он взял меня давнишнее, помолчал. – Я не могу, как бы понятая искать не могло быть и непосредственно произносит этого отдельного, не исполнено, я сделал для всякого сделать и все таки не имел ни малейшего величика

epoch 5 done
e: 6, s: 18432000:  Альшавых, будто от него, чтобы они стояли у него нашего времени, как всегда бывает с горячихи мелючных людей, князь Андрей.
«Я был у меня есть хозяив и темная сущность – скороется эта душу еще что то писать».
– Да на этою ничтожной деньга! А мне очнулась, ты пошел, а ему нужно не было, и потому, что я могу сделать добро… Потому идти не нужны, а вы и… – И вдруг нисколько. – Вы пожилает, – говорил князь, стараясь в бритах говорило только одно, что было выпускает их известием.
– Все это так страны?
Очень други
e: 6, s: 19584000:  Вмолчине и горно сказал, что она подумала, – прибавила княжна сказать всё, как будто он нжемон поговорил с собственной образом, войны была направлена более совсем под гору, от кого то, предполагая, сообщить его на свой косый хлеб от угадика видиантов и, чувствует этот стоявший поступить на действительности и добродетельного. Наташа не верила этого, и его присутствовать и от этого немецкая смела, но что этой мы не понравительный им

e: 12, s: 36864000:  Жинимах и какой был ясный. Потом Борис, с свлицанию, покраснел.
– Я нынче так, как сказал вам, напишен в коленку гостю, – сказал он, выговаривая свою презрительную красоту у древковской друг, – кричал он своим молодецким.
Князь Андрей вошел в кабинет с кровком князей и сверхом войск, струя будущую княжну Марью, с другим столом у ворах и бежал нынче в речи от между костров и планым волосом сражений, австрийской кобель. В середине которого название было совсем водки, чтобы быть недостанет со всеми в мирском д
e: 12, s: 38016000:  Стауну за дочерью отвечал на предвидении своей любимый денек, во время окна. В сумности он прежде не сказал без нее свои будущей женщины, с которого восторг на завтрашним одним мыслен.
– Все равно! Васильях, садитесь беспрерывными шагим, – крикнул он свое положение и улыбкой выговаривая по старой графине короле, – то я положил важнейших мундиры, которые получали друг, – подумал он. – В своей женщиной чувствуете, – приказал видеть его от меня

e: 18, s: 55296000:  Богня, прислушась, красная и описно, жена, наша пестна.
– Но вы так облималистом, – говорила Наташа с тем, чтобы слеза влюблена в него.
– Но, мой милый, не понимаю? – сказала она, – не ведя она во сне вдруг, чтоб он успокоится.
– Неужели вам придется, – сказал он.
– Ну, прощай, граф, – заседал князь Ипполит, – часовом и великому сюба, милый господ, никли, но и тогда, как я уверена. Но всё это значение? Напротив, я понимаю, но звуки, что нужно, равно. – И карета, задумался, повторил своим планом в лице своег
e: 18, s: 56448000:  Веречьский кружкам и самому сердцу носковым вопросом о помощах.
– Как же, ваше величество, вот треплетесь.
– Нет.
– Как только войдут верх ранен, за каждым рядом позволял богатых не только что памятный бавальных, новый с голосом подбегают в сапеднюю улыбку завоев, когда перебегают его личные глаза смотрели, и он, не ответся, проговорил с солдатами, спрашивавшим об лесу. Никто из этого видно было, как это что то странно занималось. Он ушел из

In [25]:
print(generate_characters(2048, prompt='C', pick_top_chars=3))

Cоrrrennt au malgal [оставление величества].] – проворчав один, отвечая на слова Наташи, сердитые крики и саней. Вера, в то время как она на другой день приехал в Петербург, она видела его лицо, она вздрогнула, показывая толку со строгом голосу, перебирая, уперся к солдатскую серую шля, как будто всё был при себе и передавляя его партизанского полков, неопредения, с которым он провелил в неизвестии, не отдавал приказание, но он через несколько человек, был сам своих детей. Как только плечами они стояли на восток, у бокаоновина думал о том, что он сказал, было скучное и прекрасное лицо.
Князь Андрей понял, что это будет, необыкновенно не в силах удержаться от совершенного человека. И все были сомнительны, он был в возможности жители народа встречает присутствия.
Отравление эти понятия не истребить его, она с некоторою обращаться к ней, от строгие усилиях и расспрашивали его о после населенные духа и привычными кое что они сейчас и презрительно пока, и ее измученных лиц, состоивших в себ

In [24]:
session.close()